In [1]:
def lookup(byte):
    x = byte >> 4
    y = byte & 15
    return int(AES_SBOX[x][y],16)


def reverse_lookup(byte):
    x = byte >> 4
    y = byte & 15
    return int(REV_AES_SBOX[x][y],16)


def multiply_2(value):
    res = value << 1
    res &= 0xff
    if (value & 128) != 0:
        res = res ^ 0x1b
    return res


def multiply_3(value):
    return multiply_2(value) ^ value


def mix_columns(grid):
    new_grid = [[], [], [], []]
    for i in range(4):
        col = [grid[j][i] for j in range(4)]
        col = mix_column(col)
        for i in range(4):
            new_grid[i].append(col[i])
    return new_grid


def mix_column(column):
    res = [
        multiply_2( column[0] ) ^ multiply_3( column[1] ) ^ column[2] ^ column[3],
        multiply_2( column[1] ) ^ multiply_3( column[2] ) ^ column[3] ^ column[0],
        multiply_2( column[2] ) ^ multiply_3( column[3] ) ^ column[0] ^ column[1],
        multiply_2( column[3] ) ^ multiply_3( column[0] ) ^ column[1] ^ column[2],
    ]
    return res

def rotate_row_left(row, n=1):
    return row[n:] + row[:n]


def add_sub_key(block_grid, key_grid):
    result = []

    for i in range(4):
        result.append([])
        for j in range(4):
            result[-1].append(block_grid[i][j] ^ key_grid[i][j])
    return result


def extract_key_for_round(expanded_key, round):
    return [ row [ round * 4 : round * 4 + 4 ] for row in expanded_key]


def create_4_by_4(s):
    all = []
    for i in range(len(s)//16):
        b = s[i*16: i*16 + 16]
        grid = [[], [], [], []]
        for i in range(4):
            for j in range(4):
                grid[i].append(b[i + j*4])
        all.append(grid)
    return all


def expand_key(key, rounds):

    rcon = [[1, 0, 0, 0]]

    for _ in range(1, rounds):
        rcon.append([rcon[-1][0]*2, 0, 0, 0])
        if rcon[-1][0] > 0x80:
            rcon[-1][0] ^= 0x11b

    key_grid = create_4_by_4(key)[0]

    for round in range(rounds):
        last_column = [ row[-1] for row in key_grid ]

        last_column_rotate_step = rotate_row_left(last_column)

        last_column_sbox_step = [ lookup(b) for b in last_column_rotate_step ]

        last_column_rcon_step = [ last_column_sbox_step[i]
                                 ^ rcon[round][i] for i in range(len(last_column_rotate_step)) ]

        for r in range(4):
            key_grid[r] += bytes([last_column_rcon_step[r]
                                  ^ key_grid[r][round*4]])

        for i in range(len(key_grid)):
            for j in range(1, 4):
                key_grid[i] += bytes( [key_grid[i][ round*4 + j]
                                      ^ key_grid[i][ round*4 + j + 3] ])

    return key_grid

In [2]:
AES_SBOX = [
            [ '0x63', '0x7C', '0x77', '0x7B', '0xF2', '0x6B', '0x6F', '0xC5', '0x30', '0x01', '0x67', '0x2B', '0xFE', '0xD7', '0xAB', '0x76' ],
            [ '0xCA', '0x82', '0xC9', '0x7D', '0xFA', '0x59', '0x47', '0xF0', '0xAD', '0xD4', '0xA2', '0xAF', '0x9C', '0xA4', '0x72', '0xC0' ],
            [ '0xB7', '0xFD', '0x93', '0x26', '0x36', '0x3F', '0xF7', '0xCC', '0x34', '0xA5', '0xE5', '0xF1', '0x71', '0xD8', '0x31', '0x15' ],
            [ '0x04', '0xC7', '0x23', '0xC3', '0x18', '0x96', '0x05', '0x9A', '0x07', '0x12', '0x80', '0xE2', '0xEB', '0x27', '0xB2', '0x75' ],
            [ '0x09', '0x83', '0x2C', '0x1A', '0x1B', '0x6E', '0x5A', '0xA0', '0x52', '0x3B', '0xD6', '0xB3', '0x29', '0xE3', '0x2F', '0x84' ],
            [ '0x53', '0xD1', '0x00', '0xED', '0x20', '0xFC', '0xB1', '0x5B', '0x6A', '0xCB', '0xBE', '0x39', '0x4A', '0x4C', '0x58', '0xCF' ],
            [ '0xD0', '0xEF', '0xAA', '0xFB', '0x43', '0x4D', '0x33', '0x85', '0x45', '0xF9', '0x02', '0x7F', '0x50', '0x3C', '0x9F', '0xA8' ],
            [ '0x51', '0xA3', '0x40', '0x8F', '0x92', '0x9D', '0x38', '0xF5', '0xBC', '0xB6', '0xDA', '0x21', '0x10', '0xFF', '0xF3', '0xD2' ],
            [ '0xCD', '0x0C', '0x13', '0xEC', '0x5F', '0x97', '0x44', '0x17', '0xC4', '0xA7', '0x7E', '0x3D', '0x64', '0x5D', '0x19', '0x73' ],
            [ '0x60', '0x81', '0x4F', '0xDC', '0x22', '0x2A', '0x90', '0x88', '0x46', '0xEE', '0xB8', '0x14', '0xDE', '0x5E', '0x0B', '0xDB' ],
            [ '0xE0', '0x32', '0x3A', '0x0A', '0x49', '0x06', '0x24', '0x5C', '0xC2', '0xD3', '0xAC', '0x62', '0x91', '0x95', '0xE4', '0x79' ],
            [ '0xE7', '0xC8', '0x37', '0x6D', '0x8D', '0xD5', '0x4E', '0xA9', '0x6C', '0x56', '0xF4', '0xEA', '0x65', '0x7A', '0xAE', '0x08' ],
            [ '0xBA', '0x78', '0x25', '0x2E', '0x1C', '0xA6', '0xB4', '0xC6', '0xE8', '0xDD', '0x74', '0x1F', '0x4B', '0xBD', '0x8B', '0x8A' ],
            [ '0x70', '0x3E', '0xB5', '0x66', '0x48', '0x03', '0xF6', '0x0E', '0x61', '0x35', '0x57', '0xB9', '0x86', '0xC1', '0x1D', '0x9E' ],
            [ '0xE1', '0xF8', '0x98', '0x11', '0x69', '0xD9', '0x8E', '0x94', '0x9B', '0x1E', '0x87', '0xE9', '0xCE', '0x55', '0x28', '0xDF' ],
            [ '0x8C', '0xA1', '0x89', '0x0D', '0xBF', '0xE6', '0x42', '0x68', '0x41', '0x99', '0x2D', '0x0F', '0xB0', '0x54', '0xBB', '0x16' ]
        ]

REV_AES_SBOX = [    
            [ '0x52', '0x09', '0x6A', '0xD5', '0x30', '0x36', '0xA5', '0x38', '0xBF', '0x40', '0xA3', '0x9E', '0x81', '0xF3', '0xD7', '0xFB' ],
            [ '0x7C', '0xE3', '0x39', '0x82', '0x9B', '0x2F', '0xFF', '0x87', '0x34', '0x8E', '0x43', '0x44', '0xC4', '0xDE', '0xE9', '0xCB' ],
            [ '0x54', '0x7B', '0x94', '0x32', '0xA6', '0xC2', '0x23', '0x3D', '0xEE', '0x4C', '0x95', '0x0B', '0x42', '0xFA', '0xC3', '0x4E' ],
            [ '0x08', '0x2E', '0xA1', '0x66', '0x28', '0xD9', '0x24', '0xB2', '0x76', '0x5B', '0xA2', '0x49', '0x6D', '0x8B', '0xD1', '0x25' ],
            [ '0x72', '0xF8', '0xF6', '0x64', '0x86', '0x68', '0x98', '0x16', '0xD4', '0xA4', '0x5C', '0xCC', '0x5D', '0x65', '0xB6', '0x92' ],
            [ '0x6C', '0x70', '0x48', '0x50', '0xFD', '0xED', '0xB9', '0xDA', '0x5E', '0x15', '0x46', '0x57', '0xA7', '0x8D', '0x9D', '0x84' ],
            [ '0x90', '0xD8', '0xAB', '0x00', '0x8C', '0xBC', '0xD3', '0x0A', '0xF7', '0xE4', '0x58', '0x05', '0xB8', '0xB3', '0x45', '0x06' ],
            [ '0xD0', '0x2C', '0x1E', '0x8F', '0xCA', '0x3F', '0x0F', '0x02', '0xC1', '0xAF', '0xBD', '0x03', '0x01', '0x13', '0x8A', '0x6B' ],
            [ '0x3A', '0x91', '0x11', '0x41', '0x4F', '0x67', '0xDC', '0xEA', '0x97', '0xF2', '0xCF', '0xCE', '0xF0', '0xB4', '0xE6', '0x73' ],
            [ '0x96', '0xAC', '0x74', '0x22', '0xE7', '0xAD', '0x35', '0x85', '0xE2', '0xF9', '0x37', '0xE8', '0x1C', '0x75', '0xDF', '0x6E' ],
            [ '0x47', '0xF1', '0x1A', '0x71', '0x1D', '0x29', '0xC5', '0x89', '0x6F', '0xB7', '0x62', '0x0E', '0xAA', '0x18', '0xBE', '0x1B' ],
            [ '0xFC', '0x56', '0x3E', '0x4B', '0xC6', '0xD2', '0x79', '0x20', '0x9A', '0xDB', '0xC0', '0xFE', '0x78', '0xCD', '0x5A', '0xF4' ],
            [ '0x1F', '0xDD', '0xA8', '0x33', '0x88', '0x07', '0xC7', '0x31', '0xB1', '0x12', '0x10', '0x59', '0x27', '0x80', '0xEC', '0x5F' ],
            [ '0x60', '0x51', '0x7F', '0xA9', '0x19', '0xB5', '0x4A', '0x0D', '0x2D', '0xE5', '0x7A', '0x9F', '0x93', '0xC9', '0x9C', '0xEF' ],
            [ '0xA0', '0xE0', '0x3B', '0x4D', '0xAE', '0x2A', '0xF5', '0xB0', '0xC8', '0xEB', '0xBB', '0x3C', '0x83', '0x53', '0x99', '0x61' ],
            [ '0x17', '0x2B', '0x04', '0x7E', '0xBA', '0x77', '0xD6', '0x26', '0xE1', '0x69', '0x14', '0x63', '0x55', '0x21', '0x0C', '0x7D' ]
        ]

In [3]:
def aes_encrypt(key, data):

    pad = bytes(16 - len(data) % 16)

    if len(pad) != 16:
        data += pad

    grids = create_4_by_4(data)

    expanded_key = expand_key(key, 11)

    temp_grids = []

    round_key = extract_key_for_round(expanded_key, 0)

    for grid in grids:
        temp_grids.append(add_sub_key(grid, round_key))

    grids = temp_grids

    for round in range(1, 10):
        temp_grids = []

        for grid in grids:
            sub_bytes_step = [[lookup(val) for val in row] for row in grid]
            shift_rows_step = [rotate_row_left(
                sub_bytes_step[i], i) for i in range(4)]
            mix_column_step = mix_columns(shift_rows_step)

            round_key = extract_key_for_round(expanded_key, round)

            add_sub_key_step = add_sub_key(mix_column_step, round_key)
            temp_grids.append(add_sub_key_step)

        grids = temp_grids

    temp_grids = []
    round_key = extract_key_for_round(expanded_key, 10)

    for grid in grids:
        sub_bytes_step = [[lookup(val) for val in row] for row in grid]
        shift_rows_step = [rotate_row_left(
            sub_bytes_step[i], i) for i in range(4)]


        add_sub_key_step = add_sub_key(shift_rows_step, round_key)
        temp_grids.append(add_sub_key_step)

    grids = temp_grids

    int_stream = []
    for grid in grids:
        for column in range(4):
            for row in range(4):
                int_stream.append(grid[row][column])

    return bytes(int_stream)


def aes_decrypt(key, data):

    grids = create_4_by_4(data)
    expanded_key = expand_key(key, 11)
    temp_grids = []
    round_key = extract_key_for_round(expanded_key, 10)

    temp_grids = []

    for grid in grids:

        add_sub_key_step = add_sub_key(grid, round_key)
        shift_rows_step = [rotate_row_left(
            add_sub_key_step[i], -1 * i) for i in range(4)]
        sub_bytes_step = [[reverse_lookup(val) for val in row]
                          for row in shift_rows_step]
        temp_grids.append(sub_bytes_step)

    grids = temp_grids

    for round in range(9, 0, -1):
        temp_grids = []

        for grid in grids:
            round_key = extract_key_for_round(expanded_key, round)
            add_sub_key_step = add_sub_key(grid, round_key)

            mix_column_step = mix_columns(add_sub_key_step)
            mix_column_step = mix_columns(mix_column_step)
            mix_column_step = mix_columns(mix_column_step)
            shift_rows_step = [rotate_row_left(
                mix_column_step[i], -1 * i) for i in range(4)]
            sub_bytes_step = [
                [reverse_lookup(val) for val in row] for row in shift_rows_step]
            temp_grids.append(sub_bytes_step)

        grids = temp_grids
        temp_grids = []

    round_key = extract_key_for_round(expanded_key, 0)

    for grid in grids:
        temp_grids.append(add_sub_key(grid, round_key))

    grids = temp_grids

    int_stream = []
    for grid in grids:
        for column in range(4):
            for row in range(4):
                int_stream.append(grid[row][column])

    return bytes(int_stream)

In [4]:
h_hex = ['0x6a09e667', '0xbb67ae85', '0x3c6ef372', '0xa54ff53a', '0x510e527f', '0x9b05688c', '0x1f83d9ab', '0x5be0cd19']


K = ['0x428a2f98', '0x71374491', '0xb5c0fbcf', '0xe9b5dba5', '0x3956c25b', '0x59f111f1', '0x923f82a4',
 '0xab1c5ed5', '0xd807aa98', '0x12835b01', '0x243185be', '0x550c7dc3', '0x72be5d74', '0x80deb1fe', 
 '0x9bdc06a7', '0xc19bf174', '0xe49b69c1', '0xefbe4786', '0x0fc19dc6', '0x240ca1cc', '0x2de92c6f', 
 '0x4a7484aa', '0x5cb0a9dc', '0x76f988da', '0x983e5152', '0xa831c66d', '0xb00327c8', '0xbf597fc7', 
 '0xc6e00bf3', '0xd5a79147', '0x06ca6351', '0x14292967', '0x27b70a85', '0x2e1b2138', '0x4d2c6dfc', 
 '0x53380d13', '0x650a7354', '0x766a0abb', '0x81c2c92e', '0x92722c85', '0xa2bfe8a1', '0xa81a664b', 
 '0xc24b8b70', '0xc76c51a3', '0xd192e819', '0xd6990624', '0xf40e3585', '0x106aa070', '0x19a4c116', 
 '0x1e376c08', '0x2748774c', '0x34b0bcb5', '0x391c0cb3', '0x4ed8aa4a', '0x5b9cca4f', '0x682e6ff3', 
 '0x748f82ee', '0x78a5636f', '0x84c87814', '0x8cc70208', '0x90befffa', '0xa4506ceb', '0xbef9a3f7', 
 '0xc67178f2']

def translate(message):
    charcodes = [ord(c) for c in message]
    bytes = []
    for char in charcodes:
        bytes.append(bin(char)[2:].zfill(8))
    bits = []
    for byte in bytes:
        for bit in byte:
            bits.append(int(bit))
    return bits


def chunker(bits, chunk_length=8):
    chunked = []
    for b in range(0, len(bits), chunk_length):
        chunked.append(bits[b:b+chunk_length])
    return chunked

def fillZeros(bits, length=8, endian='LE'):
    l = len(bits)
    if endian == 'LE':
        for i in range(l, length):
            bits.append(0)
    else: 
        while l < length:
            bits.insert(0, 0)
            l = len(bits)
    return bits

def preprocessMessage(message):
    bits = translate(message)
    length = len(bits)
    message_len = [int(b) for b in bin(length)[2:].zfill(64)]
    if length < 448:
        bits.append(1)
        bits = fillZeros(bits, 448, 'LE')
        bits = bits + message_len
        return [bits]
    elif 448 <= length <= 512:
        bits.append(1)
        bits = fillZeros(bits, 1024, 'LE')
        bits[-64:] = message_len
        return chunker(bits, 512)
    else:
        bits.append(1)
        while (len(bits)+64) % 512 != 0:
            bits.append(0)
        bits = bits + message_len
        return chunker(bits, 512)


def initializer(values):
    binaries = [bin(int(v, 16))[2:] for v in values]
    words = []
    for binary in binaries:
        word = []
        for b in binary:
            word.append(int(b))
        words.append(fillZeros(word, 32, 'BE'))
    return words

def b2Tob16(value):
    value = ''.join([str(x) for x in value])
    binaries = []
    for d in range(0, len(value), 4):
        binaries.append('0b' + value[d:d+4])
        hexes = ''
    for b in binaries:
        hexes += hex(int(b ,2))[2:]
    return hexes

def isTrue(x): return x == 1

def if_(i, y, z): return y if isTrue(i) else z

def and_(i, j): return if_(i, j, 0)
def AND(i, j): return [and_(ia, ja) for ia, ja in zip(i,j)] 

def not_(i): return if_(i, 0, 1)
def NOT(i): return [not_(x) for x in i]

def xor(i, j): return if_(i, not_(j), j)
def XOR(i, j): return [xor(ia, ja) for ia, ja in zip(i, j)]

def xorxor(i, j, l): return xor(i, xor(j, l))
def XORXOR(i, j, l): return [xorxor(ia, ja, la) for ia, ja, la, in zip(i, j, l)]

def maj(i,j,k): return max([i,j,], key=[i,j,k].count)

def rotr(x, n): return x[-n:] + x[:-n]

def shr(x, n): return n * [0] + x[:-n]

def add(i, j):
    length = len(i)
    sums = list(range(length))
    c = 0
    for x in range(length-1,-1,-1):
        sums[x] = xorxor(i[x], j[x], c)
        c = maj(i[x], j[x], c)
    return sums 

def sha256(message): 
    k = initializer(K)
    h0, h1, h2, h3, h4, h5, h6, h7 = initializer(h_hex)
    chunks = preprocessMessage(message)
    for chunk in chunks:
        w = chunker(chunk, 32)
        for _ in range(48):
            w.append(32 * [0])
        for i in range(16, 64):
            s0 = XORXOR(rotr(w[i-15], 7), rotr(w[i-15], 18), shr(w[i-15], 3) ) 
            s1 = XORXOR(rotr(w[i-2], 17), rotr(w[i-2], 19), shr(w[i-2], 10))
            w[i] = add(add(add(w[i-16], s0), w[i-7]), s1)
        a = h0
        b = h1
        c = h2
        d = h3
        e = h4
        f = h5
        g = h6
        h = h7
        for j in range(64):
            S1 = XORXOR(rotr(e, 6), rotr(e, 11), rotr(e, 25) )
            ch = XOR(AND(e, f), AND(NOT(e), g))
            temp1 = add(add(add(add(h, S1), ch), k[j]), w[j])
            S0 = XORXOR(rotr(a, 2), rotr(a, 13), rotr(a, 22))
            m = XORXOR(AND(a, b), AND(a, c), AND(b, c))
            temp2 = add(S0, m)
            h = g
            g = f
            f = e
            e = add(d, temp1)
            d = c
            c = b
            b = a
            a = add(temp1, temp2)
        h0 = add(h0, a)
        h1 = add(h1, b)
        h2 = add(h2, c)
        h3 = add(h3, d)
        h4 = add(h4, e)
        h5 = add(h5, f)
        h6 = add(h6, g)
        h7 = add(h7, h)
    digest = ''
    for val in [h0, h1, h2, h3, h4, h5, h6, h7]:
        digest += b2Tob16(val)
    return digest


In [7]:
import time
import hmac
import hashlib
import os

def generate_key():
    return os.urandom(32)

def encrypt(plaintext, key):
    timestamp = int(time.time()).to_bytes(8, 'big')
    iv = os.urandom(16)

    ciphertext = aes_encrypt(key,plaintext)

    # Generate HMAC using a separate key
    text = plaintext.decode()
    hmac_digest = sha256(text)
    hstring = hmac_digest.encode('utf-8')
    #mac = hmac.new(hmac_key, iv + ciphertext, hashlib.sha256).digest()
    #print(ciphertext)
    #print(hstring)
    #print(bytes.fromhex(hex_sha256_hash))
    return iv + timestamp + ciphertext + hstring
    
def decrypt(ciphertext, key):
    iv = ciphertext[:16]
    timestamp = int.from_bytes(ciphertext[16:24], 'big')
    encrypted_data = ciphertext[24:-64]
    hstring = ciphertext[-64:]
    #print(iv,encrypted_data,timestamp,hstring)
    decrypted_data = aes_decrypt(key, encrypted_data)
    #    decrypted_data = aes_decrypt(key, encrypted_data)
    d=decrypted_data.replace(b'\x00', b'')
    checkhash = sha256(d.decode())
    if (hstring.decode() != checkhash):
        raise ValueError("HMAC verification failed. Data may have been tampered.")
        
    if not is_valid_iv(iv):
        raise ValueError("Invalid IV")
    # Verify timestamp with an extended time window
    current_time = int(time.time())
    if not (current_time - 60 <= timestamp <= current_time + 60):
        raise ValueError("Invalid timestamp")
    #print(decrypted_data)
    return decrypted_data

# Example usage
key = generate_key()
message = b"Hello, this is a Fernet-like encryption example."

encrypted_data = encrypt(message, key)
print("Encrypted data:", encrypted_data)

decrypted_data = decrypt(encrypted_data, key)
print("Decrypted data:", decrypted_data)


Encrypted data: b'\xfb\xdc\xf5\xb9U1#"\x7f\xfb>\x8eE\xfd\xc0\xc7\x00\x00\x00\x00e\x85\xf8\xde\xfe\xcd\xba\xeepO\xc5\xb3n\xb6nknm#v*\x08}\xed\x03{b\xa6\xd2\x87\x13V\xa8\xffY4\x8a\xea\x99uh]o\xe4\'\x9f\x12\x86\x85\\\xc7E4fc25ab4c3c78d21366838851564668d98f73ad4dfbde9a24c89cd58ca7b2c16'
Decrypted data: b'Hello, this is a Fernet-like encryption example.'


In [6]:
def is_valid_iv(iv):
    return len(iv) == 16

In [8]:
# Paste the SHA-256 implementation and example usage here

# Example usage
message_to_send = b'Hello, this is a message.'
# Generate SHA-256 hash
hash_result = sha256(message_to_send.decode())
print("Generated SHA-256 hash:", hash_result)
print(len(hash_result))

Generated SHA-256 hash: d7c7ae15be0eb652fd8c8fbab5c85ea072e31ea0e71b6e9ad1363d81f9739e17
64


In [9]:
m = "yo"

In [10]:
m.encode()

b'yo'

In [11]:
import os
def generate_key():
    return os.urandom(32)

In [12]:
generate_key()

b"\xf7\xaauJ\xa7>\xe9'q\xb1i2{\x9f\x08(\x85'G\x98+}pA\x99\x19f\xdbkg\xaf&"

In [17]:
key = b"\xf7\xaauJ\xa7>\xe9'q\xb1i2{\x9f\x08(\x85'G\x98+}pA\x99\x19f\xdbkg\xaf&"


In [25]:
str = 'strrr'

In [27]:
str.encode()

b'strrr'